In [1]:
strain = "BY4741"
paralog = "FKS1"
# Chain identifier in the PDB
chain = "/F"

hotspots = ["HS1", "HS2", "HS3"]
compound = "caspofungin"

In [2]:
# Parameters
strain = "R1158"
paralog = "FKS2"
chain = "#2 /A"
hotspots = ["HS1", "HS2"]
compound = "none"


# Notebook to generate defattr files for ChimeraX

These files are later imported in ChimeraX to plot DMS data on protein structures

## Import libraries

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

## Enter constants

In [4]:
# Color palette
palette = "#F1F1F1:#E4C1D9:#D691C1:#C75DAB"

## Retrieve all classified data

In [5]:
list_df = []

for hs in hotspots:
    locus = "-".join([paralog, hs])
    hs_df = pd.read_csv(f"../heatmap_data/{strain}_{locus}_{compound}.csv")
    hs_df["Hotspot"] = hs
    list_df.append(hs_df)

df = pd.concat(list_df, ignore_index=True)

## Aggregate

In [6]:
attrdf = (
    df.groupby("aa_pos")[["sensres"]]
    .agg(lambda x: x[x.str.contains("resistant")].count() / len(x))
    .reset_index()
)
attrdf["pos"] = attrdf["aa_pos"].apply(lambda x: f"{chain}:{str(int(x))}")
attrdf["empty_col"] = ""
attrdf

,aa_pos,sensres,pos,empty_col
0,659,0.0,#2 /A:659,
1,660,0.0,#2 /A:660,
2,661,0.0,#2 /A:661,
3,662,0.0,#2 /A:662,
4,663,0.0,#2 /A:663,
5,664,0.0,#2 /A:664,
6,665,0.0,#2 /A:665,
7,666,0.0,#2 /A:666,
8,1372,0.0,#2 /A:1372,
9,1373,0.0,#2 /A:1373,


## Convert to defattr file

In [7]:
custom_header = f"#\nattribute: res_{compound}\nmatch mode: 1-to-1\nrecipient: residues\n"
with open(f"../defattr/{strain}_{paralog}_{compound}_defattr.txt", "w") as fp:
    fp.write(custom_header)

attrdf[["empty_col", "pos", "sensres"]].to_csv(
    f"../defattr/{strain}_{paralog}_{compound}_defattr.txt",
    sep="\t",
    header=None,
    index=None,
    mode="a",
)

cmd = f"color byattribute res_{compound} range 0, 1 palette {palette}"
print("Command lines to enter in ChimeraX")
print(f'open "../defattr/{strain}_{paralog}_{compound}_defattr.txt" format defattr')
print(cmd)

Command lines to enter in ChimeraX
open "../defattr/R1158_FKS2_none_defattr.txt" format defattr
color byattribute res_none range 0, 1 palette #F1F1F1:#E4C1D9:#D691C1:#C75DAB


In [8]:
# Example of custom color key
# Here, proportion of resistant mutants
print(
    f'key {palette.split(":")[0]}:0% '
    + " ".join([f"{c}:" for c in palette.split(":")[1:-1]])
    + f' {palette.split(":")[-1]}:100%'
)

key #F1F1F1:0% #E4C1D9: #D691C1: #C75DAB:100%
